# Import Library

In [41]:
import os
from dotenv import load_dotenv

# Vector DB
from qdrant_client import QdrantClient

# Embedding Model
from langchain_ollama import OllamaEmbeddings

# Large Language Model
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import ChatPromptTemplate
import transformers
import torch

# Document Retrieval

### Embedding Model

In [3]:
def get_embedding_model(model_name):
    '''
    Args:
        model_name (str): Name of the model to be used for embeddings
    
    Returns:
        embeddings: Embedding model object
    '''

    embeddings = OllamaEmbeddings(
        model=model_name
    )
    return embeddings

def encode_text(embeddings, text):
    '''
    Args:
        embeddings (OllamaEmbeddings): Embedding model object
        text (str): Text to be encoded

    Returns:
        np.array: Encoded vector for the input text
    '''
    
    return embeddings.embed_query(text)

### Retrieval

In [29]:
def get_chunks(vector, client, collection_name):
    '''
    Args:
        query (str): Query text
        embeddings (OllamaEmbeddings): Embedding model object
        client (QdrantClient): Qdrant client object
        collection_name (str): Name of the collection to search for chunks
    Returns:
        list: List of chunks
    '''
    
    chunks = client.query_points(
        collection_name=collection_name,
        query=vector,
        limit=3
    )
    chunks = chunks.points
    return chunks

def get_content_list(chunks):
    '''
    Args:
        chunks (list): List of ScoredPoint
    Returns:
        list: List of content
    '''
    return [chunk.payload['text'] for chunk in chunks]

def get_source_list(chunks):
    '''
    Args:
        chunks (list): List of ScoredPoint
    Returns:
        list: List of sources
    '''
    return [chunk.payload['metadata']['source'] for chunk in chunks]

In [28]:
client = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))
collection_name = "cvd_collection"
embeddings = get_embedding_model('nomic-embed-text')

query = "Apa itu angin duduk?"
query_vector = encode_text(embeddings, query)

chunks = get_chunks(query_vector, client, collection_name)

content_list = get_content_list(chunks)
source_list = get_source_list(chunks)

for content in content_list:
    print(content)

Topik: Angin Duduk, Subtopik: Gejala Angin Duduk 
 Gejala utama angin duduk adalah nyeri dada yang terasa seperti ditindih atau ditekan benda berat. Nyeri yang timbul akibat angin duduk dapat menyebar ke bagian tubuh lain, seperti leher, lengan, bahu, punggung, rahang, dan gigi. Pada wanita, terkadang nyeri dada dapat terasa seperti ditusuk benda tajam.
Beberapa gejala lain yang dapat menyertai nyeri dada pada angin duduk atau angina adalah:
- Keringat dingin
- Mual
- Pusing
- Lemas
- Sesak napas
Gejala angin duduk lebih sering timbul saat beraktivitas dan mereda atau hilang jika beristirahat atau meminum obat. Angin duduk jenis ini disebut angin duduk stabil.
Pada kasus tertentu, angin duduk tidak hilang meski sudah beristirahat dan minum obat, atau timbul saat sedang beristirahat. Angin duduk jenis ini disebut dengan angin duduk tidak stabil.

Topik: Angin Duduk, Subtopik: Pengertian Angin Duduk 
 Angina pectoris atau angin duduk ditandai dengan nyeri dada yang seperti tertindih, ter

# Generating Response

### Formulating a Prompt

In [33]:
def generate_prompt(content_list, query):
    '''
    Args:
        content_list (list): List of content
    Returns:
        str: Prompt for the user
    '''
    prompt = "Berdasarkan konteks-konteks berikut:\n"
    for content in content_list:
        prompt += f"{content}\n"
    prompt += "Jawab pertanyaan ini sesuai dengan konteks di atas: " + query
    return prompt

In [34]:
prompt = generate_prompt(content_list, query)

print(prompt)

Berdasarkan konteks-konteks berikut:
Topik: Angin Duduk, Subtopik: Gejala Angin Duduk 
 Gejala utama angin duduk adalah nyeri dada yang terasa seperti ditindih atau ditekan benda berat. Nyeri yang timbul akibat angin duduk dapat menyebar ke bagian tubuh lain, seperti leher, lengan, bahu, punggung, rahang, dan gigi. Pada wanita, terkadang nyeri dada dapat terasa seperti ditusuk benda tajam.
Beberapa gejala lain yang dapat menyertai nyeri dada pada angin duduk atau angina adalah:
- Keringat dingin
- Mual
- Pusing
- Lemas
- Sesak napas
Gejala angin duduk lebih sering timbul saat beraktivitas dan mereda atau hilang jika beristirahat atau meminum obat. Angin duduk jenis ini disebut angin duduk stabil.
Pada kasus tertentu, angin duduk tidak hilang meski sudah beristirahat dan minum obat, atau timbul saat sedang beristirahat. Angin duduk jenis ini disebut dengan angin duduk tidak stabil.

Topik: Angin Duduk, Subtopik: Pengertian Angin Duduk 
 Angina pectoris atau angin duduk ditandai dengan n

### Generating LLM Response

In [42]:


hf_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

model_id = "meta-llama/Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

llm_response = pipeline("Hey how are you doing today?")

print(llm_response)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.1-8B.
403 Client Error. (Request ID: Root=1-67727048-5e7d13ee43324e6c149f9d3f;f1e6febd-1f4c-4e8f-b564-aa73ab614530)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.1-8B to ask for access.